In [4]:
import pandas as pd
import numpy as np
import pickle
import json

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [5]:
sims_file = '../01_GeneratePureSpectrums/sims_01.csv'
res_info_folder = '../01_GeneratePureSpectrums/ResInfo/res_info_'
sims_df = pd.read_csv(sims_file)
print(sims_df)
pickle_files = '../01_GeneratePureSpectrums/specs/'+sims_df['filename']+'.pkl'

   soil_resolution         function    id                      filename
0            7x7x7               Si  1000               7x7x7_Si_001000
1            7x7x7               Al  1001               7x7x7_Al_001001
2            7x7x7                H  1002                7x7x7_H_001002
3            7x7x7               Na  1003               7x7x7_Na_001003
4            7x7x7                O  1004                7x7x7_O_001004
5            7x7x7               Fe  1005               7x7x7_Fe_001005
6            7x7x7               Mg  1006               7x7x7_Mg_001006
7            7x7x7                C  1007                7x7x7_C_001007
8            7x7x7             SiO2  1008             7x7x7_SiO2_001008
9            7x7x7            Al2O3  1009            7x7x7_Al2O3_001009
10           7x7x7              H2O  1010              7x7x7_H2O_001010
11           7x7x7             Na2O  1011             7x7x7_Na2O_001011
12           7x7x7            Fe2O3  1012            7x7x7_Fe2O3

In [ ]:
filenames = []
n_90 = []
n_95 = []
n_99 = []
n_100 = []
n_total = []

for i in range(len(sims_df)):

    pickle_file = pickle_files[i]
    soil_resolution = sims_df['soil_resolution'][i]
    res_info_file = res_info_folder + str(soil_resolution) + '.json'
    res_info = json.load(open(res_info_file, 'r'))
    midpoints = np.array(res_info['midpoints'])
    detector_tally_ids = [int(r) for r in res_info['detector_tally_ids']]
    cell_ids = [int(r) for r in res_info['cell_ids']]
    filename = sims_df['filename'][i]
    i_bins, i_spectrums= pickle.load(open(pickle_file, 'rb'))
    detector_energy_bins = i_bins[0][-2]
    detector_spec = i_spectrums[0]
    causal_bins = [str(bin)[:len(str(cell_ids[0]))] for bin in i_bins[4][2]]
    causal_bins = (np.array(causal_bins)).astype(int).tolist()
    causal_energy_bins = i_bins[4][-2]
    causal_spec = i_spectrums[4]
    new_shape = np.array(causal_spec.shape)
    new_shape[2] = len(cell_ids)
    new_array = np.zeros(new_shape)
    for j in range(len(cell_ids)):
        if cell_ids[j] in causal_bins:
            new_array[:, :, j] = causal_spec[:, :, causal_bins.index(cell_ids[j])]
    full_causal_spec = new_array
    cell_heating = full_causal_spec[0, 0, :, 0, 0, 0, :, 0]
    total_soil_heating = np.sum(cell_heating, axis=0)
    cells_over_total_heating = np.nan_to_num(np.divide(cell_heating, total_soil_heating), copy=False, nan=0.0, posinf=None, neginf=None)
    cells_over_total_heating = cells_over_total_heating/np.sum(cells_over_total_heating)
    cell_portion = np.sum(cells_over_total_heating, axis=1)

    sorted_indicies = np.argsort(cell_portion)[::-1]
    sorted_vals = (cell_portion)[sorted_indicies]
    def percent_cloud(sorted_indicies, sorted_vals, threshold=0.90):
        total = np.sum(sorted_vals)
        cumulative_sum = np.cumsum(sorted_vals)
        percent = cumulative_sum / total
        return sorted_indicies[percent < threshold]
    cell_portion = np.sum(cells_over_total_heating, axis=1)


    filenames.append(filename)
    n_90.append(len(percent_cloud(sorted_indicies, sorted_vals, threshold=.90)))
    n_95.append(len(percent_cloud(sorted_indicies, sorted_vals, threshold=.95)))
    n_99.append(len(percent_cloud(sorted_indicies, sorted_vals, threshold=.99)))
    n_100.append(len(percent_cloud(sorted_indicies, sorted_vals, threshold=1.0)))
    n_total.append(len(sorted_indicies))

results_df = pd.DataFrame({
    'filename': filenames,
    'n_90': n_90,
    'n_95': n_95,
    'n_99': n_99,
    'n_100': n_100,
    'n_total': n_total
})
results_df.to_csv('n_cells_over_total_heating.csv', index=False)
print(results_df)

                        filename  n_90  n_95  n_99  n_100  n_total
0                7x7x7_Si_001000    27    48   118    343      343
1                7x7x7_Al_001001    25    45   104    315      343
2                 7x7x7_H_001002    34    42    52    343      343
3                7x7x7_Na_001003    76   119   205    316      343
4                 7x7x7_O_001004    67    83   115    343      343
5                7x7x7_Fe_001005     8    15    48    343      343
6                7x7x7_Mg_001006    30    51   124    316      343
7                 7x7x7_C_001007    23    37    84    308      343
8              7x7x7_SiO2_001008    22    38    89    307      343
9             7x7x7_Al2O3_001009    12    21    54    275      343
10              7x7x7_H2O_001010    33    53   113    343      343
11             7x7x7_Na2O_001011    30    51   117    313      343
12            7x7x7_Fe2O3_001012    11    18    50    287      343
13              7x7x7_MgO_001013    17    29    74    343     

/tmp/ipykernel_4018425/2306235724.py:37: RuntimeWarning: invalid value encountered in divide
  cells_over_total_heating = np.nan_to_num(np.divide(cell_heating, total_soil_heating), copy=False, nan=0.0, posinf=None, neginf=None)


In [32]:
import math

In [33]:
math.factorial

<function math.factorial(x, /)>

In [37]:
# 2 nCr 22 calulatuiom using numpy:
(math.factorial(22) // (math.factorial(2) * math.factorial(22 - 2)))/22


10.5